# Import Library

In [1]:
import sqlite3
import pandas as pd
import unittest
import logging
from datetime import datetime

# Data Loading

## Import the data from Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
db_path = '/content/drive/MyDrive/subscriber-pipeline-starter-kit/dev/cademycode.db'

## Connect the data to sqlite3

In [4]:
conn = sqlite3.connect(db_path)

In [5]:
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql_query(query, conn)

dataframes = {}
for table_name in tables['name']:
    query = f"SELECT * FROM {table_name};"
    df = pd.read_sql_query(query, conn)
    dataframes[table_name] = df

In [6]:
conn.close()

## Retrieve the data as CSV format

In [7]:
output_directory = '/content/'
for table_name, df in dataframes.items():
    csv_file_path = f"{output_directory}/{table_name}.csv"
    df.to_csv(csv_file_path, index=False)
    print(f"Table '{table_name}' has been saved as '{csv_file_path}'")

Table 'cademycode_students' has been saved as '/content//cademycode_students.csv'
Table 'cademycode_courses' has been saved as '/content//cademycode_courses.csv'
Table 'cademycode_student_jobs' has been saved as '/content//cademycode_student_jobs.csv'


In [8]:
df1 = pd.read_csv('/content/cademycode_students.csv')
df2 = pd.read_csv('/content/cademycode_courses.csv')
df3 = pd.read_csv('/content/cademycode_student_jobs.csv')

# Data Preprocessing

## Handling Missing Values

In [9]:
# Check missing values in cademycode_students table
df1.isna().sum()

uuid                        0
name                        0
dob                         0
sex                         0
contact_info                0
job_id                      5
num_course_taken          251
current_career_path_id    471
time_spent_hrs            471
dtype: int64

In [10]:
# Check missing values in cademycode_courses table
df2.isna().sum()

career_path_id       0
career_path_name     0
hours_to_complete    0
dtype: int64

In [11]:
# Check missing values in cademycode_student_jobs table
df3.isna().sum()

job_id          0
job_category    0
avg_salary      0
dtype: int64

In [12]:
df1.dropna(inplace=True)

There are various missing values in the cademycode_students table, so rows that have missing values are deleted. The deletion technique is used because this technique is the fastest compared to other techniques.

## Change the data type & Rename the data

In [13]:
# Check the data type & data name from cademycode_students table
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4293 entries, 0 to 4999
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   uuid                    4293 non-null   int64  
 1   name                    4293 non-null   object 
 2   dob                     4293 non-null   object 
 3   sex                     4293 non-null   object 
 4   contact_info            4293 non-null   object 
 5   job_id                  4293 non-null   float64
 6   num_course_taken        4293 non-null   float64
 7   current_career_path_id  4293 non-null   float64
 8   time_spent_hrs          4293 non-null   float64
dtypes: float64(4), int64(1), object(4)
memory usage: 335.4+ KB


In [14]:
# Check the data type & data name from cademycode_courses table
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   career_path_id     10 non-null     int64 
 1   career_path_name   10 non-null     object
 2   hours_to_complete  10 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 368.0+ bytes


In [15]:
# Check the data type & data name from cademycode_student_jobs table
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   job_id        13 non-null     int64 
 1   job_category  13 non-null     object
 2   avg_salary    13 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 440.0+ bytes


In [16]:
df1['current_career_path_id'] = df1['current_career_path_id'].astype(int)
df1['job_id'] = df1['job_id'].astype(int)

The data type is changed because ID is generally an integer type.

In [17]:
df1 = df1.rename(columns={'current_career_path_id': 'career_path_id'})

The data name is changed so that tables can be joined.

## Merge the data

In [18]:
df4 = pd.merge(df1, df2, on='career_path_id')
mdf = pd.merge(df4, df3, on='job_id')

All tables are merged to make the data simpler to process.

# Exploratory Data Analysis (EDA)

In [19]:
mdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6022 entries, 0 to 6021
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   uuid               6022 non-null   int64  
 1   name               6022 non-null   object 
 2   dob                6022 non-null   object 
 3   sex                6022 non-null   object 
 4   contact_info       6022 non-null   object 
 5   job_id             6022 non-null   int64  
 6   num_course_taken   6022 non-null   float64
 7   career_path_id     6022 non-null   int64  
 8   time_spent_hrs     6022 non-null   float64
 9   career_path_name   6022 non-null   object 
 10  hours_to_complete  6022 non-null   int64  
 11  job_category       6022 non-null   object 
 12  avg_salary         6022 non-null   int64  
dtypes: float64(2), int64(5), object(6)
memory usage: 658.7+ KB


The data has 6022 rows and 13 columns.

In [20]:
mdf.describe()

,uuid,job_id,num_course_taken,career_path_id,time_spent_hrs,hours_to_complete,avg_salary
count,6022.000000,6022.000000,6022.000000,6022.000000,6022.000000,6022.000000,6022.000000
mean,2487.651943,4.095151,7.464796,5.460478,11.530297,21.738459,91940.551312
std,1443.328905,1.871582,4.609516,2.874791,7.608912,6.334095,29858.351075
min,1.000000,1.000000,0.000000,1.000000,0.000000,12.000000,10000.000000
25%,1237.250000,3.000000,4.000000,3.000000,5.340000,18.000000,66000.000000
50%,2503.500000,4.000000,7.000000,5.000000,10.620000,20.000000,86000.000000
75%,3717.750000,5.000000,12.000000,8.000000,16.790000,27.000000,110000.000000
max,5000.000000,8.000000,15.000000,10.000000,35.980000,35.000000,135000.000000


* The number of courses taken is between 0 to 15 with an average of 7.
* The time spent (hours) is between 0 to 35 with an average of 11.
* The hours to complete is between 12 to 35 with an average of 21.
* The average salary is between 10000 to 135000 with an average of 91000.